In [1]:
#importing libraries
import pandas as pd
import numpy as np

#reading csv for training
dataset = pd.read_csv('Social_Network_Ads.csv')
header = ['UserID', 'Gender', 'Age', 'EstimatedSalary','Purchased']

#matrix of feature, most of the datasets will have last column as dependant variable and first columns as features"""
x = dataset.iloc[:, :-1].values
#Dependant variable vector"""
y = dataset.iloc[:, -1].values
print(x, "\n");
# print(dataset.head(10));

[[15624510 'Male' 19.0 19000.0]
 [15810944 'Male' 35.0 20000.0]
 [15668575 'Female' 26.0 43000.0]
 ...
 [15654296 'Female' 50.0 20000.0]
 [15755018 'Male' 36.0 33000.0]
 [15594041 'Female' 49.0 36000.0]] 



In [2]:
print(y)

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 0 0 0 1 0 0 0 1 0 1
 1 1 0 0 1 1 0 1 1 0 1 1 0 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 0 1
 1 0 1 1 0 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0
 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 0 0 1 1 0 1 0
 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1
 1 1 0 1 0 1 0 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1]


## Taking Care of Missing data
1) We can also ignore observation by deleting it, this will work if we have 1% missing data. 2) If there is a lot of missing data, we need to handle it by replacing missing data with an average of all the columns in which data is missing.

We can replace it by using median, most frequent used value, average value which is most relavant.

Applying this imputer object to the matrix of features
Fit method - will exactly connect this imputer object to the matrix of features.It will look at the missing values in salary and age columns, and will compute average of salary but will not replace this value. So we need another method,imputer.transform().

In [3]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(x[:, 2:4])
x[:, 2:4] = imputer.transform(x[:, 2:4])

In [4]:
print(x)
print(len(x))

[[15624510 'Male' 19.0 19000.0]
 [15810944 'Male' 35.0 20000.0]
 [15668575 'Female' 26.0 43000.0]
 ...
 [15654296 'Female' 50.0 20000.0]
 [15755018 'Male' 36.0 33000.0]
 [15594041 'Female' 49.0 36000.0]]
400


## Node Class

In [5]:
class Question:
    #class to divide the available dataset by asking question

    def __init__(self,column = {},value={}):
       
        self.column = column
        self.value = value

    def match(self, example):
     
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
      #to print question
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [6]:
def partition(rows, question):
    #data is divided into true rows and false rows based on how it satisfies the question/node
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [7]:
def is_numeric(value):
    #Test if a value is numeric
    return isinstance(value, int) or isinstance(value, float)

In [8]:
def gini(rows):
   #calculate gini impurity for rows - to determine which separation is best,we need a way to measure and compare impurity
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [9]:
def info_gain(left, right, current_uncertainty):
    #calculation of information gain -The uncertainty of the starting node, minus the weighted impurity of
    #two child nodes.
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [10]:
def find_best_split(rows):
    #finding the best question by iterating over every feature and calculating the information gain
    
    best_info_gain = 0  # keep track of the best information gain
    best_node_question = None  # keep train of the feature that produced it
    current_uncertainty = gini(rows)
    
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_info_gain:
                best_info_gain, best_node_question = gain, question

    return best_info_gain, best_node_question
   

In [11]:
class Leaf:
   
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [12]:
class Decision_Node:
      def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [13]:
def build_tree(rows):
    # Partitioning the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # if no further info gain
        # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

   
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [14]:
def print_tree(node, spacing=""):
      # If we reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=.2, random_state=41)
#print(X_train)
#my_tree = build_tree(training_data)

In [16]:
def class_counts(rows):
    #Counts the number of each type of occurence in a dataset
    counts = {} 
    for row in rows:
        # label is our last column which is the target
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [17]:
dataset = np.concatenate((X_train,Y_train[:,None]),axis=1)
my_tree = build_tree(dataset)
print(len(dataset))

320


In [18]:
print_tree(my_tree)

Is Age >= 43.0?
--> True:
  Is EstimatedSalary >= 39000.0?
  --> True:
    Is Age >= 51.0?
    --> True:
      Is UserID >= 15794566?
      --> True:
        Is EstimatedSalary >= 114000.0?
        --> True:
          Predict {0: 1}
        --> False:
          Predict {1: 1}
      --> False:
        Is Age >= 59.0?
        --> True:
          Is Age >= 60.0?
          --> True:
            Predict {1: 5}
          --> False:
            Is EstimatedSalary >= 76000.0?
            --> True:
              Is UserID >= 15688172?
              --> True:
                Predict {1: 2}
              --> False:
                Predict {0: 1}
            --> False:
              Predict {0: 1}
        --> False:
          Predict {1: 19}
    --> False:
      Is EstimatedSalary >= 86000.0?
      --> True:
        Is UserID >= 15705298?
        --> True:
          Predict {1: 9}
        --> False:
          Is UserID >= 15638646?
          --> True:
            Is EstimatedSalary >= 96000.0?
   

In [19]:
def classify(row, node):
    # If we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [20]:
classify(dataset[0], my_tree)

{1: 21}

In [24]:
def print_leaf(counts):
    #Print the predictions at a leaf.
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs
    #return total

In [28]:
dataset_test = np.concatenate((X_test,Y_test[:,None]),axis=1)
dataset_test
print(len(dataset_test))


80


In [38]:
actuals = []

predicted =[]

for row in dataset_test:
    print ("Actual: %s. Predicted: %s" %
             (row[-1], print_leaf(classify(row, my_tree))))

    actuals.append(row[-1])
    my_dict = print_leaf(classify(row, my_tree))
    key_list = list(my_dict.keys())
    predicted.append(key_list)
    
#converting to 1D array so that we get only the first value of classify()
arr = np.array(predicted)
flat_array = arr.flatten()

Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {1: '100%'}
Actual: 1. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Pre

In [39]:
from sklearn.metrics import accuracy_score
#to get the accuracy score
accuracy_score(actuals, flat_array)

0.875